# 其他工具：Grab、PyQuery


* 利用 Grab 套件的存取 HTML 資源
* 利用 PyQuery 套件的解析 HTML 格式


## 作業目標

將之前用 requests + beatifulsoup 實作的方式，改寫成 grab + pyquery，並且比較有哪些地方不同。



## requests + BeautifulSoup


In [91]:
import requests
import os

from bs4 import BeautifulSoup
from PIL import Image

url = 'https://www.ptt.cc/bbs/Beauty/M.1556291059.A.75A.html'
resp = requests.get(url, cookies={'over18': '1'})
soup = BeautifulSoup(resp.text)
# 決定要儲存的資料夾
output_dir = 'downloads'

# 假如資料夾不存在就新增一個資料夾
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 定位所有圖片的 tag
image_tags = soup.find_all('a',rel='nofollow')
for img_tag in image_tags:
    # 取得所有圖片在第三方服務的 id
    if 'imgur' not in img_tag['href']:
        continue
    print(img_tag['href'])
    img_id = img_tag['href'].split('/')[-1]
    # 組合圖片而非網站的網址
    img_url = 'https://i.imgur.com/{}.jpg'.format(img_id)
    # 對圖片送出請求
    with requests.get(img_url, stream=True) as r:
        r.raise_for_status()
        # 檢查圖片副檔名
        img = Image.open(r.raw)
        img_savename = '{outdir}/{img_id}.{img_ext}'.format(outdir=output_dir,img_id=img_id,img_ext=img.format.lower())
        img.save(img_savename)
        print('Save image as {}'.format(img_savename))

https://imgur.com/Cgb5oo1
Save image as downloads/Cgb5oo1.jpeg
https://imgur.com/MgjHY4f
Save image as downloads/MgjHY4f.jpeg
https://imgur.com/yeoRtAs
Save image as downloads/yeoRtAs.jpeg
https://imgur.com/wsauNEW
Save image as downloads/wsauNEW.jpeg
https://imgur.com/IjiETcs
Save image as downloads/IjiETcs.jpeg


## Grab + PyQuery

In [123]:
from grab import Grab
from pyquery import PyQuery as pq
from PIL import Image, ImageFile
import io

url = 'https://www.ptt.cc/bbs/Beauty/M.1556291059.A.75A.html'
g = Grab(user_agent='grab')
cookies = {'over18':'1'}
resp = g.go(url, cookies=cookies)
#print(resp.body)
doc = pq(resp.body)
# 決定要儲存的資料夾
output_dir = 'downloads1'

# 假如資料夾不存在就新增一個資料夾
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 定位所有圖片的 tag
images = []
for i in doc.items('a'):
    if 'imgur' not in i.attr('href'):
        continue
    img_id = i.attr('href').split('/')[-1]
    if img_id not in images:
        images.append(img_id)
    else:
        continue
        
    # 組合圖片而非網站的網址
    img_url = 'https://i.imgur.com/{}.jpg'.format(img_id)
    # 對圖片送出請求；同時需要重新建立Grab()，才不會抓不到資料
    gb = Grab()
    r = gb.go(img_url)
    tempfile = '{}.jpg'.format(img_id)
    open(tempfile, 'wb').write(r.body)
    
    img = Image.open(tempfile)
    img_savename = '{outdir}/{img_id}.{img_ext}'.format(outdir=output_dir,img_id=img_id,img_ext=img.format.lower())
    img.save(img_savename)
    print('Save image as {}'.format(img_savename))



Save image as downloads1/Cgb5oo1.jpeg
Save image as downloads1/MgjHY4f.jpeg
Save image as downloads1/yeoRtAs.jpeg
Save image as downloads1/wsauNEW.jpeg
Save image as downloads1/IjiETcs.jpeg
